Code to extract and save $\Delta F / F$ for whole making whole brain maps

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import glob
from pathlib import Path
import pickle
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from janelia_core.dataprocessing.dataset import ROIDataset
from janelia_core.utils.data_saving import append_ts

from keller_zlatic_vnc.data_processing import extract_transitions
from keller_zlatic_vnc.data_processing import match_annotation_subject_to_volume_subject
from keller_zlatic_vnc.data_processing import read_raw_transitions_from_excel
from keller_zlatic_vnc.data_processing import recode_beh

## Parameters go here

In [3]:
ps = dict()

# Location of excel file specifying where the data for each experiment is saved relative to the base folder
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'

# Location of excel file specifying transition information 
ps['trans_file'] = r'A:\projects\keller_vnc\data\extracted_dff_v2\transition_list.xlsx'

# Subfolder containing the dataset for each subject
ps['dataset_folder'] = 'extracted'

# Base folder where datasets are stored 
ps['dataset_base_folder'] =r'K:\\SV4'

# Data to calculate Delta F/F for in each dataset
ps['f_ts_str'] = 'f_1_5_5'
ps['bl_ts_str'] = 'bl_1_5_5'

ps['n_smps'] = 3 # Number of samples to take before and after a manipulation 

# Parameters for calculating dff
ps['background'] = 100
ps['ep'] = 20

# Location to save results
ps['save_folder'] = r'A:\projects\keller_vnc\results\whole_brain_stats'
ps['save_str'] = 'dff_1_5_5_with_ep'

## Read in excel file specifying location of datasets

In [4]:
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

## Read in transition information

In [5]:
trans = read_raw_transitions_from_excel(ps['trans_file'], adjust_frame_index=True)
trans = recode_beh(trans, 'beh_before')
trans = recode_beh(trans, 'beh_after')

## Helper function to calculate $\Delta F/F$

In [6]:
def calc_dff(f, b, background=ps['background'], ep=ps['ep']):
    return (f-b)/(b-background+ep)

## Extract dff along with behavior information from each dataset

In [ ]:
n_datasets = len(data_locs)
event_annots = [None]*n_datasets

unique_trans_subjs = trans['sample_id'].unique()
n_trans_subjs = len(unique_trans_subjs)

matched_trans_subjs = np.zeros(n_trans_subjs)
matched_data_subjs = np.zeros(n_datasets)

for d_i in range(n_datasets):
    
    # ===============================================================================
    # Find the appropriate annotations for this experiment
    data_main_folder = data_locs['Main folder'][d_i]
    data_sub_folder = data_locs['Subfolder'][d_i]

    # Parse the subject from the subfolder string
    match_ind = match_annotation_subject_to_volume_subject(data_main_folder, data_sub_folder, unique_trans_subjs)

    if match_ind is not None:

        # If we can find this subject in the annotations, make sure we haven't matched to it before
        if matched_trans_subjs[match_ind] == True:
            raise(RuntimeError('Found a transition subject we already matched to: d_i=' + str(d_i)))
        if matched_data_subjs[d_i] == True:
            raise(RuntimeError('Found a volume subject we already matched to.'))
            
        matched_trans_subjs[match_ind] = True
        matched_data_subjs[d_i] = True
         
        # ===============================================================================
        # Now that we know we have a unique match, we do the rest of our processing

        # Read in the data for this subject
        dataset_path = (Path(ps['dataset_base_folder']) / data_main_folder / data_sub_folder / 
                        Path(ps['dataset_folder']) / '*.pkl')
        dataset_file = glob.glob(str(dataset_path))[0]

        # Load the dataset
        with open(dataset_file, 'rb') as f:
            dataset = ROIDataset.from_dict(pickle.load(f))
        
        # Calculate dff
        f=dataset.ts_data[ps['f_ts_str']]['vls'][:]
        b=dataset.ts_data[ps['bl_ts_str']]['vls'][:]
        dff = calc_dff(f=f, b=b)
        
        # Extract dff for each event
        sample_id = unique_trans_subjs[match_ind]
        event_rows = trans['sample_id'] == sample_id
        sample_events = copy.deepcopy(trans[event_rows])
        n_sample_events = len(sample_events)
        dff_before = [None]*n_sample_events
        dff_during = [None]*n_sample_events
        dff_after = [None]*n_sample_events
        
        for e_i in range(n_sample_events):
            m_start = sample_events['Manipulation Start'].to_numpy()[e_i]
            m_end = sample_events['Manipulation End'].to_numpy()[e_i]
            
            dff_before[e_i] = np.mean(dff[m_start-ps['n_smps']:m_start,:], axis=0)
            dff_during[e_i] = np.mean(dff[m_start:m_end+1,:], axis=0)
            dff_after[e_i] = np.mean(dff[m_end+1:m_end+ps['n_smps']+1,:], axis=0)
            
        sample_events['dff_before'] = dff_before
        sample_events['dff_during'] = dff_during
        sample_events['dff_after'] = dff_after
            
        # Save results
        event_annots[d_i] = sample_events
    
        # Give user some feedback
        print('Done processing dataset ' + str(d_i + 1) + ' of ' + str(n_datasets) + '.')

In [ ]:
event_annots = pd.concat(event_annots)

In [ ]:
# See what subjects we have transitions for but no registerd volumes
ignored_trans_subjs = unique_trans_subjs[np.logical_not(matched_trans_subjs)]

# See what subjects we have registered volumes for but no transitions
ignored_vol_subjs = data_locs[np.logical_not(matched_data_subjs)]

In [ ]:
## Save results
rs = dict()
rs['ps'] = ps
rs['event_annots'] = event_annots
rs['ignored_trans_subjs'] = ignored_trans_subjs
rs['ignored_vol_subjs'] = ignored_vol_subjs

save_name = append_ts(ps['save_str']) + '.pkl'
save_path = Path(ps['save_folder']) / save_name
with open(save_path, 'wb') as f:
    pickle.dump(rs, f)
    
print('Saved results to: ' + str(save_path))